In [1]:
!pip install keras

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 377 kB 858 kB/s eta 0:00:01


In [17]:
from keras.layers import Dense, Input, LSTM, Embedding,Dropout,Bidirectional,Flatten
from keras.layers import*
from keras.engine.topology import Layer
from keras import backend as K
import os
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd
import numpy as np
import keras
import h5py
from keras.optimizers import RMSprop


class attention_layers(Layer):
    def __init__(self, **kwargs):
        super(attention_layers, self).__init__(**kwargs)

    def build(self,inputshape):
        assert len(inputshape) == 3
        #以下是keras的自己开发工作
        self.W = self.add_weight(name='attr_weight',
                                 shape=(inputshape[1],inputshape[1]),
                                 initializer='uniform',
                                 trainable=True)
        self.b = self.add_weight(name='attr_bias',
                                 shape=(inputshape[1],),
                                 initializer='uniform',
                                 trainable=True)
        super(attention_layers, self).build(inputshape)

    def call(self,inputs):
        print(inputs)
        print('w',self.W)
        print('b',self.b)
        x = K.permute_dimensions(inputs, (0, 2, 1))
        a = K.softmax(K.tanh(K.dot(x, self.W) + self.b))

        print('a',a)
        print('x',x)
        outputs = K.permute_dimensions(a*x, (0, 2, 1))
        outputs = K.sum(outputs, axis=1)
        return  outputs

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[2]

def attention_3d_block(inputs):
    # input_dim = int(inputs.shape[2])
    print('inputs',inputs)
    a = Permute((2, 1))(inputs)
    print('a',a)
    a = Dense(200, activation='softmax')(a)
    print('Dense', a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    print('a_probs', a_probs)
    # output_attention_mul = merge([inputs, a_probs], name='attention_mul', mode='mul')
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul')
    print('output_attention_mul', output_attention_mul)
    return output_attention_mul
    
    
def get_filename_for_saving(save_dir):
    return os.path.join(save_dir,
                        "lstm_bi_nodrop_3d_attention_ad_age_softsign2{val_loss:.3f}-{val_accuracy:.3f}-{epoch:03d}-{loss:.3f}-{accuracy:.3f}.hdf5")


def buid_model():
    # LSTM 模型
    print('lstm model start...\n')

    # 标题输入：接收一个含有 200 个整数的序列，每个整数在 1 到 3812202 之间。
    main_input = Input(shape=(200,), name='main_input', dtype='int32')
    emb = Embedding(output_dim=150, input_dim=3812203, input_length=200,mask_zero = True)(main_input)
    lstm_out = Bidirectional(LSTM(50,activation='softsign',return_sequences=False))(emb)
#     lstm_out = Bidirectional(LSTM(10,return_sequences=False,activation='softsign'))(lstm_out1)
#     lstm_out = attention_3d_block(lstm_out)
#     lstm_out = GlobalAveragePooling1D()(lstm_out)
    
#     lstm_out = Bidirectional(LSTM(10,activation='softsign'))(emb)
#     attention_mul = attention_3d_block(lstm_out)
#     lstm_out = Flatten()(attention_mul)
#     lstm_out = Dropout(0.3)(lstm_out)
    
    # 堆叠全连接
#     den_x = Dense(32, activation='relu')(lstm_out)
    
#     # 辅助输入数据
#     auxiliary_input = Input(shape=(48,), name='aux_input')
#     # 将辅助输入与lstm输出连接起来
#     den_x = keras.layers.concatenate([den_x, auxiliary_input])

    # 辅助损失函数
#     auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm_out)
    # 主输出
    
    main_output = Dense(10, activation='softmax', name='main_output')(lstm_out)

    # 定义一个具有两个输入输出的模型
    model = keras.models.Model(inputs=[main_input],#,auxiliary_input],
                               outputs=[main_output])  # 这里的输入输出顺序与fit时一致就好
    
    opt = RMSprop(lr=0.01,  clipnorm=1.0)
    model.compile(optimizer=opt,
                  sample_weight_mode='None',#"temporal",
                  loss={'main_output': 'categorical_crossentropy'},
                 metrics=['accuracy'])

    print('lstm model geted...\n')
    return model


def data_load():
    print('loading data ... \n')

    with h5py.File('../word_train_ad.h5', 'r') as f:
        data = np.array(f.get('word_data'))

    label = pd.read_csv('../../../train_preliminary/user.csv').sort_values(by=['user_id'])

    train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.2, random_state=2020)

    train_y_age = train_y['age'].values - 1
    train_y_age = keras.utils.np_utils.to_categorical(train_y_age, num_classes=10)
    train_y_gender = train_y['gender'].values - 1

    test_y_age = test_y['age'].values - 1
    test_y_age = keras.utils.np_utils.to_categorical(test_y_age, num_classes=10)
    test_y_gender = test_y['gender'].values - 1

    print('get data ... \n')

    return train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender

def load_data_sta():
    data_sta = pd.read_csv('../../../train_feat_statisc.csv').sort_values(by = 'user_id')
    
    label = pd.read_csv('../../../train_preliminary/user.csv').sort_values(by=['user_id'])
    data_sta = label.merge(data_sta, on='user_id', how='left').sort_values(by='user_id')
    data_sta.fillna(0)
    data_sta = data_sta.drop(['Unnamed: 0','user_id','gender','age'], axis=1)

    train_x, test_x= train_test_split(data_sta, test_size=0.2, random_state=2020)
    return train_x, test_x

In [18]:
model = buid_model()

lstm model start...

inputs Tensor("bidirectional_7/concat:0", shape=(?, 200, 20), dtype=float32)
a Tensor("permute_5/transpose:0", shape=(?, 20, 200), dtype=float32)
Dense Tensor("dense_2/truediv:0", shape=(?, 20, 200), dtype=float32)
a_probs Tensor("attention_vec_1/transpose:0", shape=(?, 200, 20), dtype=float32)
output_attention_mul Tensor("attention_mul_1/mul:0", shape=(?, 200, 20), dtype=float32)
lstm model geted...



In [19]:
print(model.summary())

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_7 (Embedding)         (None, 200, 15)      57183045    main_input[0][0]                 
__________________________________________________________________________________________________
bidirectional_7 (Bidirectional) (None, 200, 20)      2080        embedding_7[0][0]                
__________________________________________________________________________________________________
permute_5 (Permute)             (None, 20, 200)      0           bidirectional_7[0][0]            
____________________________________________________________________________________________

In [15]:
train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender = data_load()

# train_x_sta, test_x_sta = load_data_sta()

loading data ... 

get data ... 



In [20]:
print('lstm model fit...\n')
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=get_filename_for_saving(''),
    save_best_only=False)
stopping = keras.callbacks.EarlyStopping(patience=8)
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.0001)

# with h5py.File('../train_data_weight.h5', 'r') as f:
#         weight = np.array(f.get('weight'))
# train_w, test_w= train_test_split(weight, test_size=0.2, random_state=2020)

# train_w = np.squeeze(train_w)
# t_w = 10/np.log(train_w)


model.fit({'main_input': train_x },#,'aux_input': train_x_sta},
          {'main_output': train_y_age},
          epochs=100,
          batch_size=256,
#           class_weight = {0:0.698, 1:0.520, 2:0.671, 3:0.698, 4:0.945,5:0.890, 6:0.986, 7:1. , 8:0.808,9:0.657},
#           class_weight = {0:1.,1:2.159}
#           sample_weight = t_w,
          validation_data=({'main_input': test_x},#,'aux_input': test_x_sta},
                           {'main_output': test_y_age}),
          callbacks=[checkpointer, reduce_lr, stopping])

lstm model fit...


Train on 720000 samples, validate on 180000 samples
Epoch 1/100
499456/720000 [===================>..........] - ETA: 10:41 - loss: 1.9588 - accuracy: 0.2475

KeyboardInterrupt: 

In [ ]:

# pre = model.predict(test_x,verbose=1)

# #评估结果
# from sklearn.metrics import confusion_matrix, classification_report
# y_ = np.reshape(np.argmax(test_y,axis=1),[-1])
# pre_ = np.reshape(np.argmax(pre, axis=1),[-1])
# #每个类的各项指标
# cm = confusion_matrix(y_, pre_)
# # np.set_printoptions(precision=3)
# cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
# print(cm_normalized)
# print(classification_report(y_, pre_))